# Desafio 1 Hand Talk - classificação de frases por setor

Dada uma base de dados com frases classificadas em contexto/setores (atividades econômicas) específicos, seu objetivo, caso deseje aceitar, é criar um modelo de classificação para classificar frases nos setores anotados.

Logo abaixo temos a base de dados que você deverá usar para treinar seu modelo. Ela contém **521** frases classificadas entre os setores: `finanças, educação, indústrias, varejo, orgão público`.

Importante avisar que na base de dados existem frases classificadas com um único ou múltiplos setores. O caso de múltiplos setores ocorre devido a frase poder possuir um vocabulário cujo os termos pertencem a setores diferentes. Exemplo:

> **Curso de Técnico em Segurança do Trabalho por 32x R$ 161,03.**


É uma frase que poderia ser classificada nos setores `educação` e `finanças`.

Logo, seu modelo deve tratar a possibilidade da frase possuir multiplas classificações!

### Importanto bibliotecas e pacotes

In [1]:
import pandas as pd
import numpy as np
import re

# ML packages
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import accuracy_score,hamming_loss,classification_report

### train/test dataset split
from sklearn.model_selection import train_test_split

# TFIDF (vetorizador)
from sklearn.feature_extraction.text import TfidfVectorizer

# Multi Label packages
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN

### Ler dataset.csv e observar alguns detalhes dos dados (descrição, distribuição, valores nulos e necessidade de tratamento).

In [2]:
df = pd.read_csv("phrases.csv")
display(df)

df.isnull().sum()

,sentence,category
0,"Auxílio-Doença Previdenciário, Benefícios em E...",orgão público
1,"PAGAR TODAS AS CONTAS EM ATRASO R$1.290,90.",finanças
2,Então encontraremos na próxima aula.,educação
3,Veja os resultados da categoria de ofertas do ...,indústrias
4,"Além disso, a embalagem é reutilizável e 100% ...","indústrias,varejo"
...,...,...
516,"Selecione o local de estudo, curso sem encontr...",educação
517,ESTUDANTES DA REDE MUNICIPAL VOLTAM ÀS AULAS E...,"educação,orgão público"
518,Empresas e órgãos públicos,orgão público
519,DGE – Departamento de Gestão Estratégica Metas...,orgão público


sentence    0
category    0
dtype: int64

In [3]:
df.info()

df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  521 non-null    object
 1   category  521 non-null    object
dtypes: object(2)
memory usage: 8.3+ KB


,sentence,category
count,521,521
unique,513,14
top,Câmbio do novo carro Tracker 2022.,orgão público
freq,2,140


### * Existem registros repetidos no dataset!

In [4]:
df2 = df[df["sentence"] == "Câmbio do novo carro Tracker 2022."]
display(df2)

print(df["category"].unique())

,sentence,category
413,Câmbio do novo carro Tracker 2022.,indústrias
495,Câmbio do novo carro Tracker 2022.,indústrias


['orgão público' 'finanças' 'educação' 'indústrias' 'indústrias,varejo'
 'varejo' 'finanças,varejo' 'educação,orgão público' 'educação,indústrias'
 'finanças,indústrias' 'finanças,orgão público' 'varejo,indústrias'
 'indústrias,orgão público' 'educação,finanças']


In [5]:
df3 = df[df.duplicated(subset=['sentence'],keep=False)]
display(df3)

,sentence,category
60,"Valor Boleto R$ 888,00 no cartão de crédito em...",varejo
110,Acompanhe as ações da SPTrans que contribuem p...,orgão público
117,PROGRAMA DE GOVERNO PAULO SERRA,orgão público
184,"Valor Boleto R$ 888,00 no cartão de crédito em...",varejo
186,Núcleo de Estudos em Marketing e Pesquisa de O...,educação
338,Acompanhe as ações da SPTrans que contribuem p...,orgão público
345,Selecione o tipo de documento; Documentos Pess...,educação
385,PROGRAMA DE GOVERNO PAULO SERRA,orgão público
410,Veja como foi a trajetória de debates do Proje...,orgão público
413,Câmbio do novo carro Tracker 2022.,indústrias


### Será q esse desequilíbrio das targets vai ser um problema?
*Vou ignorar por hora.

In [6]:
df.groupby("category").count()

,sentence
category,
educação,107
"educação,finanças",2
"educação,indústrias",5
"educação,orgão público",9
finanças,54
"finanças,indústrias",3
"finanças,orgão público",4
"finanças,varejo",13
indústrias,89


### Partindo pro tratamento das categorias do "target" - encoding.
*Tive problemas com algumas ferramentas já prontas de encoding, então "codei na mão" um encoding rápido dos valores das targets dos registros.

In [7]:
df4 = pd.DataFrame(data=None, index = df.index, columns= ["sentence", "varejo", "orgão público", "indústrias", "educação", "finanças"])

for i, item in enumerate(df["category"]):
    if "varejo" in df["category"][i]:
        df4["varejo"][i] = 1
        if "," in df["category"][i]:
            if "orgão público" in df["category"][i]:
                df4["orgão público"][i] = 1
            elif "indústrias" in df["category"][i]:
                df4["indústrias"][i] = 1
            elif "educação" in df["category"][i]:
                df4["educação"][i] = 1
            elif "finanças" in df["category"][i]:
                df4["finanças"][i] = 1
            else:
                pass
        else:
            pass
    elif "orgão público" in df["category"][i]:
        df4["orgão público"][i] = 1
        if "," in df["category"][i]:
            if "varejo" in df["category"][i]:
                df4["varejo"][i] = 1
            elif "indústrias" in df["category"][i]:
                df4["indústrias"][i] = 1
            elif "educação" in df["category"][i]:
                df4["educação"][i] = 1
            elif "finanças" in df["category"][i]:
                df4["finanças"][i] = 1
            else:
                pass
        else:
            pass
    elif "indústrias" in df["category"][i]:
        df4["indústrias"][i] = 1
        if "," in df["category"][i]:
            if "varejo" in df["category"][i]:
                df4["varejo"][i] = 1
            elif "orgão público" in df["category"][i]:
                df4["orgão público"][i] = 1
            elif "educação" in df["category"][i]:
                df4["educação"][i] = 1
            elif "finanças" in df["category"][i]:
                df4["finanças"][i] = 1
            else:
                pass
        else:
            pass
    elif "educação" in df["category"][i]:
        df4["educação"][i] = 1
        if "," in df["category"][i]:
            if "varejo" in df["category"][i]:
                df4["varejo"][i] = 1
            elif "orgão público" in df["category"][i]:
                df4["orgão público"][i] = 1
            elif "indústrias" in df["category"][i]:
                df4["indústrias"][i] = 1
            elif "finanças" in df["category"][i]:
                df4["finanças"][i] = 1
            else:
                pass
        else:
            pass
    elif "finanças" in df["category"][i]:
        df4["finanças"][i] = 1
        if "," in df["category"][i]:
            if "varejo" in df["category"][i]:
                df4["varejo"][i] = 1
            elif "orgão público" in df["category"][i]:
                df4["orgão público"][i] = 1
            elif "indústrias" in df["category"][i]:
                df4["indústrias"][i] = 1
            elif "educação" in df["category"][i]:
                df4["educação"][i] = 1
            else:
                pass
        else:
            pass

df4 = df4.fillna(0)
df4["sentence"] = df["sentence"]
display(df4)

,sentence,varejo,orgão público,indústrias,educação,finanças
0,"Auxílio-Doença Previdenciário, Benefícios em E...",0,1,0,0,0
1,"PAGAR TODAS AS CONTAS EM ATRASO R$1.290,90.",0,0,0,0,1
2,Então encontraremos na próxima aula.,0,0,0,1,0
3,Veja os resultados da categoria de ofertas do ...,0,0,1,0,0
4,"Além disso, a embalagem é reutilizável e 100% ...",1,0,1,0,0
...,...,...,...,...,...,...
516,"Selecione o local de estudo, curso sem encontr...",0,0,0,1,0
517,ESTUDANTES DA REDE MUNICIPAL VOLTAM ÀS AULAS E...,0,1,0,1,0
518,Empresas e órgãos públicos,0,1,0,0,0
519,DGE – Departamento de Gestão Estratégica Metas...,0,1,0,0,0


## Tratamento frases!

### Definindo função para remover acentos

In [8]:
# char codes: https://unicode-table.com/en/#basic-latin
accent_map = {
    u'\u00c0': u'A',
    u'\u00c1': u'A',
    u'\u00c2': u'A',
    u'\u00c3': u'A',
    u'\u00c4': u'A',
    u'\u00c5': u'A',
    u'\u00c6': u'A',
    u'\u00c7': u'C',
    u'\u00c8': u'E',
    u'\u00c9': u'E',
    u'\u00ca': u'E',
    u'\u00cb': u'E',
    u'\u00cc': u'I',
    u'\u00cd': u'I',
    u'\u00ce': u'I',
    u'\u00cf': u'I',
    u'\u00d0': u'D',
    u'\u00d1': u'N',
    u'\u00d2': u'O',
    u'\u00d3': u'O',
    u'\u00d4': u'O',
    u'\u00d5': u'O',
    u'\u00d6': u'O',
    u'\u00d7': u'x',
    u'\u00d8': u'0',
    u'\u00d9': u'U',
    u'\u00da': u'U',
    u'\u00db': u'U',
    u'\u00dc': u'U',
    u'\u00dd': u'Y',
    u'\u00df': u'B',
    u'\u00e0': u'a',
    u'\u00e1': u'a',
    u'\u00e2': u'a',
    u'\u00e3': u'a',
    u'\u00e4': u'a',
    u'\u00e5': u'a',
    u'\u00e6': u'a',
    u'\u00e7': u'c',
    u'\u00e8': u'e',
    u'\u00e9': u'e',
    u'\u00ea': u'e',
    u'\u00eb': u'e',
    u'\u00ec': u'i',
    u'\u00ed': u'i',
    u'\u00ee': u'i',
    u'\u00ef': u'i',
    u'\u00f1': u'n',
    u'\u00f2': u'o',
    u'\u00f3': u'o',
    u'\u00f4': u'o',
    u'\u00f5': u'o',
    u'\u00f6': u'o',
    u'\u00f8': u'0',
    u'\u00f9': u'u',
    u'\u00fa': u'u',
    u'\u00fb': u'u',
    u'\u00fc': u'u'
}

def accent_remove (m):
  return accent_map[m.group(0)]


### Removendo acentos!

*talvez se forem muitos dados este processo pode levar algum tempo.

In [9]:
frases = list(df4["sentence"])
for i, sen in enumerate(frases):
    string_velha = sen
    string_nova = re.sub(u'([\u00C0-\u00FC])', accent_remove, string_velha.encode().decode('utf-8'))
    df4["sentence"][i] = string_nova
df5 = df4
display(df5)

C:\Users\leozindobololo\AppData\Local\Temp\ipykernel_12792\2944292842.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4["sentence"][i] = string_nova


,sentence,varejo,orgão público,indústrias,educação,finanças
0,"Auxilio-Doenca Previdenciario, Beneficios em E...",0,1,0,0,0
1,"PAGAR TODAS AS CONTAS EM ATRASO R$1.290,90.",0,0,0,0,1
2,Entao encontraremos na proxima aula.,0,0,0,1,0
3,Veja os resultados da categoria de ofertas do ...,0,0,1,0,0
4,"Alem disso, a embalagem e reutilizavel e 100% ...",1,0,1,0,0
...,...,...,...,...,...,...
516,"Selecione o local de estudo, curso sem encontr...",0,0,0,1,0
517,ESTUDANTES DA REDE MUNICIPAL VOLTAM AS AULAS E...,0,1,0,1,0
518,Empresas e orgaos publicos,0,1,0,0,0
519,DGE – Departamento de Gestao Estrategica Metas...,0,1,0,0,0


### Definindo função para fazer alguns outros preprocessamentos das frases - remover pontuação, caracteres únicos e espaços múltiplos.

In [10]:
def preprocess_text(sen):
    # Remove punctuations
    sentence = re.sub('[!-#%-.:-@]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence


In [11]:
for i, sen in enumerate(frases):
    string_nova2 = preprocess_text(sen)
    df5["sentence"][i] = string_nova2
df6 = df5
display(df6)

C:\Users\leozindobololo\AppData\Local\Temp\ipykernel_12792\1842313581.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5["sentence"][i] = string_nova2


,sentence,varejo,orgão público,indústrias,educação,finanças
0,Auxílio Doença Previdenciário Benefícios em Es...,0,1,0,0,0
1,PAGAR TODAS AS CONTAS EM ATRASO R$1 290 90,0,0,0,0,1
2,Então encontraremos na próxima aula,0,0,0,1,0
3,Veja os resultados da categoria de ofertas do ...,0,0,1,0,0
4,Além disso embalagem é reutilizável 100 recicl...,1,0,1,0,0
...,...,...,...,...,...,...
516,Selecione local de estudo curso sem encontros ...,0,0,0,1,0
517,ESTUDANTES DA REDE MUNICIPAL VOLTAM ÀS AULAS E...,0,1,0,1,0
518,Empresas órgãos públicos,0,1,0,0,0
519,DGE – Departamento de Gestão Estratégica Metas...,0,1,0,0,0


### Colocando todas as frases em letra minúscula

In [12]:
for i, sen in enumerate(frases):
    string_nova3 = sen.lower()
    df6["sentence"][i] = string_nova3
df7 = df6
display(df7)

C:\Users\leozindobololo\AppData\Local\Temp\ipykernel_12792\4015024951.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6["sentence"][i] = string_nova3


,sentence,varejo,orgão público,indústrias,educação,finanças
0,"auxílio-doença previdenciário, benefícios em e...",0,1,0,0,0
1,"pagar todas as contas em atraso r$1.290,90.",0,0,0,0,1
2,então encontraremos na próxima aula.,0,0,0,1,0
3,veja os resultados da categoria de ofertas do ...,0,0,1,0,0
4,"além disso, a embalagem é reutilizável e 100% ...",1,0,1,0,0
...,...,...,...,...,...,...
516,"selecione o local de estudo, curso sem encontr...",0,0,0,1,0
517,estudantes da rede municipal voltam às aulas e...,0,1,0,1,0
518,empresas e órgãos públicos,0,1,0,0,0
519,dge – departamento de gestão estratégica metas...,0,1,0,0,0


### Aqui considerei o tratamento do texto finalizado.
*Poderia fazer também Stemming ou Lemmatization das palavras, além da remoção de stopwords (tudo em PT-BR). Mas deixarei isso de lado, por hora.

#### *Portanto, para classificar novas frases seria sempre bom realizar esse tratamento antes do input no modelo.

## Feature Engineering

### Precisamos vetorizar as frases para o entendimento da máquina. Para isso vou utilizar o TFIDF (um algoritmo bastante comum para transformar texto em uma representação numérica, que então é usada para fazer o treinamento do modelo).

In [13]:
vectorizer = TfidfVectorizer()
vectorizer

TfidfVectorizer()

#### *O vectorizer funciona como um modelo também, então posso incluir ele em uma pipeline em conjunto com o modelo preditivo que eu desenvolver.

In [14]:
Xfeatures = vectorizer.fit_transform(df7["sentence"]).toarray()

In [15]:
Xfeatures

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.60933758, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [16]:
y = df7[["varejo", "orgão público", "indústrias", "educação", "finanças"]]

In [17]:
# Split Data 
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,y,test_size=0.2,random_state=66)

In [24]:
print(df7["sentence"].shape)
print(X_train.shape)

(521,)
(416, 2024)


# Construindo alguns modelos:

In [27]:
### Problem Transform
import skmultilearn

### Aqui vamos utilizar o BinaryRelevance do skmultilearn para transformar o problema de classificação multi-label (com N labels) em (N) problemas separados de classificação binária, todos utilizando Multinomial Naive Bayes para realizar a classificação.

## * Tive a ideia de testar o Gaussian Naive Bayes para classificar. Então vou aplicar nos mesmos modelos de Multinomial NB de antes.

#### Construindo e treinando modelo

In [28]:
binary_rel_clf = BinaryRelevance(MultinomialNB())

In [29]:
binary_rel_clf.fit(X_train,y_train)

BinaryRelevance(classifier=MultinomialNB(), require_dense=[True, True])

#### Avaliando modelo

In [30]:
# Predictions
br_prediction = binary_rel_clf.predict(X_test)

In [31]:
br_prediction

<105x5 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Column format>

#### Convertendo para arrays para observar os resultados

In [32]:
br_prediction.toarray()

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0,

#### Métricas de desempenho das previsões

* A acurácia nos indica os acertos das previsões

In [33]:
# Accuracy
accuracy_score(y_test,br_prediction)

0.1619047619047619

* E o Hamming Loss é uma métrica relacionada com os erros das predições (portanto, quanto menor, melhor).

In [34]:
# Hamming Loss
hamming_loss(y_test,br_prediction)

0.18857142857142858

#### Podemos reparar que o modelo não desempenhou nada bem. 

* O modelo não funcionou - não classificou diversas frases, como podemos observar pela maioria dos arrays apresentando somente zeros.

## Outros modelos:

Vou tentar construir algum modelo que realmente funcione e provavelmente definir um "baseline".

#### Vou definir uma função com a qual consigo construir um modelo e alterar o estimador de multi label utilizado, e ela já me retorna as métricas de performance do modelo, além das previsões para observação.

In [35]:
def build_model(model,mlb_estimator,xtrain,ytrain,xtest,ytest):
    # Instance
    clf = mlb_estimator(model)
    clf.fit(xtrain,ytrain)
    # Predictions
    clf_predictions = clf.predict(xtest)
    # Metrics
    acc = accuracy_score(ytest,clf_predictions)
    ham = hamming_loss(ytest,clf_predictions)
    result = {"accuracy:":acc,"hamming_score":ham, "predictions":clf_predictions.toarray()}
    return result

### BinaryRelevance com Gaussian NB

In [36]:
binary_rel_clf2 = build_model(GaussianNB(),BinaryRelevance,X_train,y_train,X_test,y_test)

In [37]:
binary_rel_clf2

{'accuracy:': 0.47619047619047616,
 'hamming_score': 0.13142857142857142,
 'predictions': array([[0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 1, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0,

#### Como podemos reparar, mudando o classificador para Gaussian NB a performance deste primeiro modelo (onde transformamos o problema com Binary Relevance) melhorou consideravelmente, mesmo que seja um modelo que não é muito preciso (acurácia de 0,476).

 * Mas a principal mudança é que agora o modelo classifica algumas frases com mais de um label!!!

### Classifier Chains
* Para N labels ele treina N classificadores ordenados de acordo com a Bayesian Chain Rule.

In [38]:
clf_chain_model = build_model(MultinomialNB(),ClassifierChain,X_train,y_train,X_test,y_test)

In [39]:
clf_chain_model

{'accuracy:': 0.18095238095238095,
 'hamming_score': 0.18285714285714286,
 'predictions': array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0.

#### Novamente, podemos reparar que o modelo utilizando o Multinomial NB não desempenhou nada bem.
* O modelo não funcionou - não classificou diversas frases, como podemos observar pela maioria dos arrays apresentando somente zeros.

### Classifier Chain com Gaussian NB

In [40]:
clf_chain_model2 = build_model(GaussianNB(),ClassifierChain,X_train,y_train,X_test,y_test)

In [41]:
clf_chain_model2

{'accuracy:': 0.4857142857142857,
 'hamming_score': 0.1295238095238095,
 'predictions': array([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 

#### Como podemos reparar mais uma vez, mudando o classificador para Gaussian NB a performance deste modelo (onde transformamos o problema com Classifier Chain) melhorou consideravelmente, mesmo que seja um modelo que não é muito preciso (acurácia de 0,486).

 * O modelo também classifica algumas frases com mais de um label!!! Aparentemente com o GaussianNB os modelos estão realmente funcionais.

### LabelPowerset

* Transforma um problema multi-label em um problema multi-classe com 1 classificador multi-classe treinado em todas as combinações de labels diferentes (consideradas as classes) encontradas nos dados.

In [42]:
clf_labelP_model = build_model(MultinomialNB(),LabelPowerset,X_train,y_train,X_test,y_test)

In [43]:
clf_labelP_model

{'accuracy:': 0.638095238095238,
 'hamming_score': 0.13904761904761906,
 'predictions': array([[0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0

#### A transformação com LabelPowerset resultou em uma performance bem melhor do que nos outros dois testes anteriores.
* O modelo funcionou - classificou todas as frases, com uma acurácia de 0,6381 e um hamming score de 0,1390.

* Mas com um detalhe (**!!!**) : eu esperava que o modelo funcionasse para classificação multi-label, mas ele não está classificando uma frase com mais de um label, nunca.

### Label Powerset com Gaussian NB

In [44]:
clf_labelP_model2 = build_model(GaussianNB(),LabelPowerset,X_train,y_train,X_test,y_test)

In [45]:
clf_labelP_model2

{'accuracy:': 0.6571428571428571,
 'hamming_score': 0.10857142857142857,
 'predictions': array([[0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 1, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 

##### Novamente, mudando o classificador para Gaussian NB a performance do modelo melhorou consideravelmente.

 * O modelo também classifica algumas frases com mais de um label!!! Com o GaussianNB os modelos estão realmente funcionais.
 
 * O modelo funcionou apresentou uma acurácia de 0,657 e um hamming score de 0,1085. Portanto, foi o modelo que performou melhor nos dados de teste!!!

### Então, vamos construir o mesmo modelo de classificação (com Gaussian NB, utilizando LabelPowerset) fora da função para ser o baseline / "MVP".

In [46]:
clf_LPmodel = LabelPowerset(GaussianNB())

In [47]:
clf_LPmodel.fit(X_train,y_train)

LabelPowerset(classifier=GaussianNB(), require_dense=[True, True])

In [48]:
clf_predictions = clf_LPmodel.predict(X_test)

In [49]:
acc = accuracy_score(y_test,clf_predictions)
ham = hamming_loss(y_test,clf_predictions)

result = {"accuracy:":acc,"hamming_score":ham, "predictions":clf_predictions.toarray()}

print(result)

{'accuracy:': 0.6571428571428571, 'hamming_score': 0.10857142857142857, 'predictions': array([[0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 0, 1, 0],
 

### Salvar o modelo utilizando joblib
* Também vou salvar o vetorizador!

In [50]:
import joblib

In [51]:
#save model
filename = 'text_classifier.joblib.pkl'
_ = joblib.dump(clf_LPmodel, filename, compress=9)

In [52]:
#save vectorizer
filename = 'vectorizer.joblib.pkl'
_ = joblib.dump(vectorizer, filename, compress=9)

### Criando uma subsample para testar o uso do modelo após o reload.

In [53]:
#Creating subsample for test
df5 = df4.loc[np.random.choice(df.index, 20, replace=False)]
display(df5)

,sentence,varejo,orgão público,indústrias,educação,finanças
183,material de aula - slides,0,0,0,1,0
134,"carros ofertas serviços, acessórios vendas e f...",1,0,0,0,0
305,graduação superior de tecnologia em automação ...,0,0,1,1,0
430,aposentadoria por tempo de contribuição artigo...,0,1,0,0,0
416,o limite do cartão de crédito pode ser reduzid...,0,0,0,0,1
39,microondas consul 32 litros na cor branca com ...,1,0,0,0,0
390,a embalagem é reutilizável e 100% reciclável e...,1,0,1,0,0
362,as ofertas são diferentes para cada localizaçã...,1,0,0,0,0
225,informe o seu cep para proporcionarmos uma mel...,1,0,0,0,0
242,resolva os testes dentro do tempo total.,0,0,0,1,0


### Carregando o modelo e o vetorizador e fazendo um teste do uso do modelo

In [54]:
#load
import joblib

clf2 = joblib.load("text_classifier.joblib.pkl")
clf2

LabelPowerset(classifier=GaussianNB(), require_dense=[True, True])

In [55]:
vectorizer2 = joblib.load("vectorizer.joblib.pkl")
vectorizer2

TfidfVectorizer()

In [56]:
newvectordata = vectorizer.transform(df5["sentence"]).toarray()

In [57]:
clf2_predictions = clf2.predict(newvectordata)

In [58]:
# Convert to Array  To See Result
clf2_predictions.toarray()

array([[0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0]], dtype=int64)

### Como podemos ver, o modelo está funcional e performando rasoavelmente bem. Então temos um modelo baseline e um MVP!

### Posteriormente podemos partir para a otimização / refino do modelo, buscando uma maior acurácia.

### Também, como já citado, podemos construir uma pipeline de dados com o vetorizador e o modelo para uma utilização mais prática.